In [1]:
import polars as pl

In [3]:
df = pl.read_csv('../data/train.csv', separator=';')
print(df.shape)
df.head(2)

(27251, 2)


sample,annotation
str,str
"""aa""","""[(0, 2, 'O')]"""
"""aala""","""[(0, 4, 'O')]"""


In [39]:
import json
pl.Config.set_fmt_str_lengths(100)
rows = df.rows()
rows[0]

('aa', "[(0, 2, 'O')]")

In [40]:
import json
from razdel import tokenize as rtokenize

def tokenize(text: str) -> list[str]:
    return text.split(' ')

tokens_ner = []
for text, ann in rows:
    ann = eval(ann)
    tokens = tokenize(text)
    ners = [ner for _, _, ner in ann]
    tokens_ner.append(list(zip(tokens, ners)))

In [ ]:
df

sample,annotation
str,str
"""aa""","""[(0, 2, 'O')]"""
"""aala""","""[(0, 4, 'O')]"""
"""aarcca""","""[(0, 6, 'O')]"""
"""abon""","""[(0, 4, 'O')]"""
"""abso""","""[(0, 4, 'B-BRAND')]"""
"""abtoys игруш""","""[(0, 6, 'B-BRAND'), (7, 12, 'B-TYPE')]"""
"""abtoys игрушк""","""[(0, 6, 'B-BRAND'), (7, 13, 'B-TYPE')]"""
"""abtoys игрушки""","""[(0, 6, 'B-BRAND'), (7, 14, 'B-TYPE')]"""
"""acofe""","""[(0, 5, 'O')]"""


In [49]:
from itertools import chain

pl.Config.set_tbl_rows(20)
pl.DataFrame(chain(*tokens_ner), schema=['token', 'ner'], orient='row') \
    .filter(pl.col('ner') == 'B-PERCENT') \
    .group_by('token').len() \
    .sort('len', descending=True) \
    .head(20)

token,len
str,u32
"""1""",2
"""10%""",2
"""33""",2
"""9""",2
"""20""",2
"""15%""",2
"""5%""",2
"""10""",2
"""82""",1
